# NRC-23 - Image Quality Verification by Filter   

## Notebook: Cross-matching catalogs

**Author**: Matteo Correnti, STScI Scientist II
<br>
**Created**: October, 2021
<br>
**Last Updated**: February, 2022

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
    2.2 [Plotting functions imports](#matpl_imports)<br>
3. [Import images to analyze](#data)<br>
    3.1 [Select Detector/Filter to analyze](#sel_data)<br>
    3.2 [Transform the images to data models](#transf_images)<br>
    3.3 [Load the PSF photometry catalogs](#load_cat)<br>
    3.4 [Clean the catalogs](#clean_cat)<br>
4. [Transform pixel to equatorial coordinates](#transf_coord)<br>
5. [Cross-Match the catalogs](#cross-match)<br>
6. [Diagnostic plots](#plots)<br>
    6.1[$\Delta$ RA vs $\Delta$ Dec plot (mas)](#delta_radec)<br>
    6.2[$\Delta$ X vs $\Delta$ Y plot (px)](#delta_xy)<br>
    6.3[$\Delta$ X (Y) as a function of X (Y)](#delta_pos)<br>
7. [Comparison with LMC astrometric catalog](#comp_lmc)<br>
    7.1 [Import LMC astrometric catalog](#load_lmc_cat)<br>
    7.2 [Select a sub-region coincident with our field of view](#fov_lmc)<br>
    7.3 [Cross-match with LMC astrometric catalog](#crossmatch_lmc)<br>
8. [Dither offset check](#dither)<br>
    8.1 [Cross-match the catalogs - same filter, different dither position](#crossmatch_dither)<br>
    8.2 [Convert sky coordinates to Ideal coordinates](#convert_dither)<br>
    8.3 [Plot offset and offset corrected (using header information](#offset_dither)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

This notebook shows how to cross-match the catalogs obtained from the PSF photometry and how to derive the possible filter position-offset. Some diagnostic plots are included in the notebook. We also include a quick check to verify the cross-match with the input LMC catalog and a check of the dither accuracy.

**Dependencies**:  before running this notebook it is necessary to perform PSF photometry on the images, to obtain the source positions, using the notebook `NRC-23_psf_photometry_multiproc.ipynb`.

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all the necessary Python packages and we define some plotting parameters.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import os 

import sys
import time
import copy

import glob as glob

import numpy as np

import pandas as pd

from astropy import units as u
from astropy.io import fits
from astropy.table import Table, QTable
from astropy.stats import sigma_clipped_stats

import jwst
from jwst.datamodels import ImageModel

from astropy.coordinates import SkyCoord, match_coordinates_sky

import pysiaf

### 2.2<font color='white'>-</font>Plotting function imports<a class="anchor" id="matpl_imports"></a> ###

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 20

font1 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '12'}
font2 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '20'}

In [ ]:
figures_dir = 'FIGURES/'

if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

3.<font color='white'>-</font>Import images to analyze<a class="anchor" id="data"></a>
------------------

In [ ]:
dict_images = {'NRCA1': {}, 'NRCA2': {}, 'NRCA3': {}, 'NRCA4': {}, 'NRCA5': {},
               'NRCB1': {}, 'NRCB2': {}, 'NRCB3': {}, 'NRCB4': {}, 'NRCB5': {}}

dict_filter_short = {}
dict_filter_long = {}

ff_short = []
det_short = []
det_long = []
ff_long = []
detlist_short = []
detlist_long = []
filtlist_short = []
filtlist_long = []

images_dir = '../Simulation/Pipeline_Outputs/Level2_Outputs'
images = sorted(glob.glob(os.path.join(images_dir, "*cal.fits")))

for image in images:

    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p
    
    wv = float(f[1:3])

    if wv > 24:         
        ff_long.append(f)
        det_long.append(d)

    else:
        ff_short.append(f)
        det_short.append(d)   

    detlist_short = sorted(list(dict.fromkeys(det_short)))
    detlist_long = sorted(list(dict.fromkeys(det_long)))

    unique_list_filters_short = []
    unique_list_filters_long = []

    for x in ff_short:

        if x not in unique_list_filters_short:

            dict_filter_short.setdefault(x, {})
                 
    for x in ff_long:
        if x not in unique_list_filters_long:
            dict_filter_long.setdefault(x, {})   
            
    for d_s in detlist_short:
        dict_images[d_s] = copy.deepcopy(dict_filter_short)

    for d_l in detlist_long:
        dict_images[d_l] = copy.deepcopy(dict_filter_long)

    filtlist_short = sorted(list(dict.fromkeys(dict_filter_short)))
    filtlist_long = sorted(list(dict.fromkeys(dict_filter_long)))

print("Available Detectors for SW channel:", detlist_short)
print("Available Detectors for LW channel:", detlist_long)
print("Available SW Filters:", filtlist_short)
print("Available LW Filters:", filtlist_long)

In [ ]:
for image in images:
    
    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p

    if len(dict_images[d][f]) == 0:
        dict_images[d][f] = {'images': [image]}
    else:
        dict_images[d][f]['images'].append(image)


### 3.1<font color='white'>-</font>Select detector/filter to analyze<a class="anchor" id="sel_data"></a> ###

In [ ]:
det = 'NRCB1'
filt1 = 'F115W'
filt2 = 'F200W'

### 3.2<font color='white'>-</font>Transform the images to data models<a class="anchor" id="transf_images"></a> ###

In order to assign the WCS coordinate and hence cross-match the images, we need to transform the images to DataModel. The coordinates are assigned during the step [assign_wcs](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/main.html?#using-the-wcs-interactively) step in the JWST pipeline and allow us to cross-match the different catalogs obtained for each filter.

In [ ]:
images_1 = []
images_2 = []

for i in np.arange(0, len(dict_images[det][filt1]['images']), 1):

    image_1 = ImageModel(dict_images[det][filt1]['images'][i])
    images_1.append(image_1)
        
for i in np.arange(0, len(dict_images[det][filt2]['images']), 1):

    image_2 = ImageModel(dict_images[det][filt2]['images'][i])
    images_2.append(image_2)

### 3.3<font color='white'>-</font>Load the PSF photometry catalogs<a class="anchor" id="load_cat"></a> ###

In [ ]:
# adopted reduction parameters (needed to retrieve the photometry catalogs from the right folder)

npsfs = 16
th = 10
fitshape = (11, 11)

cat_dir = 'PSF_PHOT_OUTPUT/numPSFs{}_Th{}_fitshape{}x{}'.format(npsfs, th, fitshape[0], fitshape[1])

phots_pkl_1 = sorted(glob.glob(os.path.join(cat_dir, 'phot_'+str.lower(det)+'_'+filt1+'*.pkl')))
phots_pkl_2 = sorted(glob.glob(os.path.join(cat_dir, 'phot_'+str.lower(det)+'_'+filt2+'*.pkl')))

phots_pkl_1

results_1 = []
results_2 = []

for phot_pkl_1 in phots_pkl_1:
    ph_1 = pd.read_pickle(phot_pkl_1)
    result_1 = QTable.from_pandas(ph_1)
    results_1.append(result_1)

for phot_pkl_2 in phots_pkl_2:
    ph_2 = pd.read_pickle(phot_pkl_2)
    result_2 = QTable.from_pandas(ph_2)
    results_2.append(result_2)

### 3.4<font color='white'>-</font>Clean the catalogs<a class="anchor" id="clean_cat"></a> ###

In [ ]:
results_clean_1 = []
results_clean_2 = []

for i in np.arange(0, len(images_1), 1):

    mask_1 = ((results_1[i]['x_fit'] > 0) & (results_1[i]['x_fit'] < images_1[0].shape[1]) &
                  (results_1[i]['y_fit'] > 0) & (results_1[i]['y_fit'] < images_1[0].shape[0]) &
                  (results_1[i]['flux_fit'] > 0))

    result_clean_1 = results_1[i][mask_1]

    results_clean_1.append(result_clean_1)

for i in np.arange(0, len(images_2), 1):
    
    mask_2 = ((results_2[i]['x_fit'] > 0) & (results_2[i]['x_fit'] < images_2[0].shape[1]) &
                  (results_2[i]['y_fit'] > 0) & (results_2[i]['y_fit'] < images_2[0].shape[0]) &
                  (results_2[i]['flux_fit'] > 0))

    result_clean_2 = results_2[i][mask_2]

    results_clean_2.append(result_clean_2)

4.<font color='white'>-</font>Transform pixel to equatorial coordinates<a class="anchor" id="transf_coord"></a>
------------------

We can transform the (x,y) positions from the raw, NIRCam coordinate system to the Equatorial system (ICRS) by means of the WCS information stored in the ASDF extension of our image.

<div class="alert alert-block alert-info">
    <h3><u><b>NB</b></u></h3>
    
For stage-2 images (_cal.fits_ files), the WCS information is assigned by the _assign_wcs_ step. This information is saved in an ASDF extension of the FITS file. However, image-display tools such as ds9 do not understand the ASDF encoding. For this reason, an approximation to the WCS is stored in the image header using the SIP (Simple Imaging Polynomial) convention. The SIP-based header does not provide an exact fit: it is accurate to a $\sim$0.25-pixel level and it is meant for general display purposes (see <a href="https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html">here</a> for more information).
</ul>
</div>

In [ ]:
results_final_1 = []
results_final_2 = []

for i in np.arange(0, len(images_1), 1):
    
    result_final_1 = results_clean_1[i]
    ra_1, dec_1 = images_1[i].meta.wcs(result_final_1['x_fit'], result_final_1['y_fit'])
    radec_1 = SkyCoord(ra_1, dec_1, unit='deg')
    result_final_1['radec'] = radec_1
    results_final_1.append(result_final_1)


for i in np.arange(0, len(images_2), 1):

    result_final_2 = results_clean_2[i]
    ra_2, dec_2 = images_2[i].meta.wcs(result_final_2['x_fit'], result_final_2['y_fit'])
    radec_2 = SkyCoord(ra_2, dec_2, unit='deg')
    result_final_2['radec'] = radec_2
    results_final_2.append(result_final_2)

5.<font color='white'>-</font>Cross-match the catalogs<a class="anchor" id="cross-match"></a>
------------------

We cross-match the catalogs to obtain the difference in positions between the different filters.

Stars from the two filters are associated if the distance between the matches is < 0.5 px. 
**Note**: larger discrepancies (i.e., strong effects introduced by the filters) should be easy to spot on the images. The maximum "cross-match distance" could be modified accordingly or another script should be used to determine the offset.  

We can cross-match each single catalog for the two filters:

In [ ]:
max_sep = 0.5
pixelscale = 0.031 # select the right pixelscale for SW or LW detector

matches_phot_1 = []
matches_phot_2 = []

for res1 in results_final_1:
    
    for res2 in results_final_2:
    
        match_phot_1 = Table()
        match_phot_2 = Table()
    
        idx, d2d, _ = match_coordinates_sky(res1['radec'], res2['radec'])
        sep_constraint = (d2d.arcsec / pixelscale) < max_sep
    
        match_phot_1 = res1[sep_constraint]
        match_phot_2 = res2[idx[sep_constraint]]
    
        matches_phot_1.append(match_phot_1)
        matches_phot_2.append(match_phot_2)
    

6.<font color='white'>-</font>Diagnostic plots<a class="anchor" id="plots"></a>
------------------

The plots are obtained for only 1 set of images to provide an example.

In [ ]:
def pix2arcsec_sw(x):
    return x * 0.031
def arcsec2pix_sw(x):
    return x / 0.031
def pix2arcsec_lw(x):
    return x * 0.063
def arcsec2pix_lw(x):
    return x / 0.063

### 6.1<font color='white'>-</font>$\Delta$ RA vs $\Delta$ Dec plot (mas)<a class="anchor" id="delta_radec"></a> ###

In [ ]:
cat1 = matches_phot_1[0]
cat2 = matches_phot_2[0]

plt.figure(figsize=(12, 12))

ax1 = plt.subplot(1, 1, 1)

xlim0 = -10
xlim1 = 10
ylim0 = -10
ylim1 = 10

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.set_xlabel('$\Delta$ RA (mas)', fontdict=font2)
ax1.set_ylabel('$\Delta$ Dec (mas)', fontdict=font2)
ax1.set_title(det + ' - ' +filt1+ ' vs '+ filt2, fontdict=font2)

avg_dec = np.deg2rad((cat1['radec'].dec.degree + cat2['radec'].dec.degree) / 2)
diff_ra = (u.deg * ((cat1['radec'].ra.degree - cat2['radec'].ra.degree) * np.cos(avg_dec))).to(u.mas) / (1 * u.mas)

diff_dec = (u.deg * (cat1['radec'].dec.degree - cat2['radec'].dec.degree)).to(u.mas) / (1 * u.mas)

_, med_diff_ra, sig_diff_ra = sigma_clipped_stats(diff_ra)
_, med_diff_dec, sig_diff_dec = sigma_clipped_stats(diff_dec)

ax1.scatter(diff_ra, diff_dec, s=5, color='k')
ax1.plot([0, 0], [ylim0, ylim1], color='k', lw=2, ls='--')
ax1.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax1.text(xlim0 + 0.05, ylim1 - 1.0, ' $\Delta$ RA (mas) = %5.3f $\pm$ %5.3f'
         % (med_diff_ra, sig_diff_ra), color='k', fontdict=font2)
ax1.text(xlim0 + 0.05, ylim1 - 1.75, ' $\Delta$ Dec (mas) = %5.3f $\pm$ %5.3f'
         % (med_diff_dec, sig_diff_dec), color='k', fontdict=font2)

plt.tight_layout()

filename = 'deltaRA_deltaDec_{}_{}_vs_{}.png'.format(det, filt1, filt2)

#plt.savefig(os.path.join(figures_dir, filename))

### 6.2<font color='white'>-</font>$\Delta$ X vs $\Delta$ Y plot (px)<a class="anchor" id="delta_xy"></a> ###

In [ ]:
cat1 = matches_phot_1[0]
cat2 = matches_phot_2[0]

plt.figure(figsize=(12, 12))

ax1 = plt.subplot(1, 1, 1)

xlim0 = -0.5
xlim1 = 0.5
ylim0 = -0.5
ylim1 = 0.5

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())


ax1.set_xlabel('$\Delta$ X [px]', fontdict=font2)
ax1.set_ylabel('$\Delta$ Y [px]', fontdict=font2)

ax1.set_title(det + ' - '+ filt1 + ' vs '+ filt2, fontdict=font2)

diff_x = cat1['x_fit'] - cat2['x_fit']
diff_y = cat1['y_fit'] - cat2['y_fit']

_, med_diff_x, sig_diff_x = sigma_clipped_stats(diff_x)
_, med_diff_y, sig_diff_y = sigma_clipped_stats(diff_y)

ax1.scatter(diff_x, diff_y, s=5, color='k')
ax1.plot([0, 0], [ylim0, ylim1], color='k', lw=2, ls='--')
ax1.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax1.text(xlim0 + 0.05, ylim1 - 0.05, ' $\Delta$ X (px) = %6.4f $\pm$ %5.4f'
         % (med_diff_x, sig_diff_x), color='k', fontdict=font2)
ax1.text(xlim0 + 0.05, ylim1 - 0.1, ' $\Delta$ Y (px) = %6.4f $\pm$ %5.4f'
         % (med_diff_y, sig_diff_y), color='k', fontdict=font2)


ax2 = ax1.secondary_xaxis('top', functions=(pix2arcsec_sw, arcsec2pix_sw))
ax2.set_xlabel('$\Delta$ X [arcsec]', fontdict=font2)
ay2 = ax1.secondary_yaxis('right', functions=(pix2arcsec_sw, arcsec2pix_sw))
ay2.set_ylabel('$\Delta$ Y [arcsec]', fontdict=font2)

plt.tight_layout()

filename = 'deltaX_deltaY_{}_{}_vs_{}.png'.format(det, filt1, filt2)

#plt.savefig(os.path.join(figures_dir, filename))

### 6.3<font color='white'>-</font>$\Delta$ X (Y) as a function of X (Y)<a class="anchor" id="delta_pos"></a> ###

In [ ]:
plt.figure(figsize=(14, 8))
plt.suptitle('$\Delta$ X (Y) as a function of X (Y) for filter ' + filt2 , fontsize=20)

ax1 = plt.subplot(2, 2, 1)

xlim0 = 0
xlim1 = images_1[0].shape[1]
ylim0 = -0.5
ylim1 = 0.5

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.set_xlabel('X (px)', fontdict=font2)
ax1.set_ylabel('$\Delta$ X (px)', fontdict=font2)

delta_x = cat1['x_fit'] - cat2['x_fit']
delta_y = cat1['y_fit'] - cat2['y_fit']

ax1.scatter(cat2['x_fit'], delta_x, s=3, color='k')
ax1.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax2 = plt.subplot(2, 2, 2)

xlim0 = 0
xlim1 = images_1[0].shape[0]
ylim0 = -0.5
ylim1 = 0.5

ax2.set_xlim(xlim0, xlim1)
ax2.set_ylim(ylim0, ylim1)

ax2.xaxis.set_major_locator(ticker.AutoLocator())
ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax2.yaxis.set_major_locator(ticker.AutoLocator())
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax2.set_xlabel('Y (px)', fontdict=font2)
ax2.set_ylabel('$\Delta$ X (px)', fontdict=font2)

ax2.scatter(cat2['y_fit'], delta_x, s=3, color='k')
ax2.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax3 = plt.subplot(2, 2, 3)

xlim0 = 0
xlim1 = images_1[0].shape[1]
ylim0 = -0.5
ylim1 = 0.5

ax3.set_xlim(xlim0, xlim1)
ax3.set_ylim(ylim0, ylim1)

ax3.xaxis.set_major_locator(ticker.AutoLocator())
ax3.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax3.yaxis.set_major_locator(ticker.AutoLocator())
ax3.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax3.set_xlabel('X (px)', fontdict=font2)
ax3.set_ylabel('$\Delta$ Y (px)', fontdict=font2)

ax3.scatter(cat2['x_fit'], delta_y, s=3, color='k')
ax3.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax4 = plt.subplot(2, 2, 4)

xlim0 = 0
xlim1 = images_1[0].shape[0]
ylim0 = -0.5
ylim1 = 0.5

ax4.set_xlim(xlim0, xlim1)
ax4.set_ylim(ylim0, ylim1)

ax4.xaxis.set_major_locator(ticker.AutoLocator())
ax4.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax4.yaxis.set_major_locator(ticker.AutoLocator())
ax4.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax4.set_xlabel('Y (px)', fontdict=font2)
ax4.set_ylabel('$\Delta$ Y (px)', fontdict=font2)

ax4.scatter(cat2['y_fit'], delta_y, s=3, color='k')
ax4.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

plt.tight_layout()

filename = 'deltaX_deltaY_vs_XY_{}_{}.png'.format(det, filt2)

#plt.savefig(os.path.join(figures_dir, filename))

7.<font color='white'>-</font>Comparison with LMC astrometric catalog<a class="anchor" id="comp_lmc"></a>
------------------

### 7.1<font color='white'>-</font>Import LMC astrometric catalog<a class="anchor" id="load_lmc_cat"></a> ###

In [ ]:
input_cat = '../Simulation/lmc_pointsources.cat'

cat = pd.read_csv(input_cat, header=None, sep='\s+', names=['index', 'ra_in', 'dec_in'],
                  comment='#', skiprows=7, usecols=range(0, 3))
cat.head()

### 7.2<font color='white'>-</font>Select a sub-region coincident with our field of view<a class="anchor" id="fov_lmc"></a> ###

In [ ]:
print('Total number of stars in the LMC catalog:', len(cat))

xlim0 = 0
ylim0 = 0
xlim1 = images_1[0].shape[1]
ylim1 = images_1[0].shape[0]

ra_lim0, dec_lim0 = images_1[0].meta.wcs(xlim0, ylim0)
print(ra_lim0, dec_lim0)
ra_lim1, dec_lim1 = images_1[0].meta.wcs(xlim1 - 1, ylim1 - 1)
print(ra_lim1, dec_lim1)

ra_lim = sorted([ra_lim0, ra_lim1])
dec_lim = [dec_lim0, dec_lim1]

cat_sel = cat.loc[(cat['ra_in'] > ra_lim[0]) & (cat['ra_in'] < ra_lim[1]) & (cat['dec_in'] > dec_lim[0]) & 
              (cat['dec_in'] < dec_lim[1])]

print('Number of LMC stars in the field of view:', len(cat_sel))
#cat_sel.head()

### 7.3<font color='white'>-</font>Cross-match with LMC astrometric catalog<a class="anchor" id="crossmatch_lmc"></a> ###

In [ ]:
radec_input = SkyCoord(cat_sel['ra_in'], cat_sel['dec_in'], unit='deg')
catalog1 = results_final_2[0]

idx_inp, d2d_inp, _ = match_coordinates_sky(radec_input, catalog1['radec'])
sep_constraint_inp = (d2d_inp.arcsec / pixelscale) < max_sep
match_phot_inp = Table()
match_phot_inp['radec'] = radec_input[sep_constraint_inp]
match_phot_out = catalog1[idx_inp[sep_constraint_inp]]


In [ ]:
plt.figure(figsize=(12, 12))

ax1 = plt.subplot(1, 1, 1)

xlim0 = -10
xlim1 = 10
ylim0 = -10
ylim1 = 10

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.set_xlabel('$\Delta$ RA (mas)', fontdict=font2)
ax1.set_ylabel('$\Delta$ Dec (mas)', fontdict=font2)
ax1.set_title('Input vs Output - '+ det + ' - ' + filt2, fontdict=font2)

avg_dec = np.deg2rad((match_phot_inp['radec'].dec.degree + match_phot_out['radec'].dec.degree) / 2)
diff_ra = (u.deg * ((match_phot_inp['radec'].ra.degree - 
                     match_phot_out['radec'].ra.degree) * np.cos(avg_dec))).to(u.mas) / (1 * u.mas)

diff_dec = (u.deg * (match_phot_inp['radec'].dec.degree - match_phot_out['radec'].dec.degree)).to(u.mas) / (1 * u.mas)

_, med_diff_ra, sig_diff_ra = sigma_clipped_stats(diff_ra)
_, med_diff_dec, sig_diff_dec = sigma_clipped_stats(diff_dec)

ax1.scatter(diff_ra, diff_dec, s=5, color='k')
ax1.plot([0, 0], [ylim0, ylim1], color='k', lw=2, ls='--')
ax1.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax1.text(xlim0 + 0.05, ylim1 - 1.0, ' $\Delta$ RA (mas) = %5.3f $\pm$ %5.3f'
         % (med_diff_ra, sig_diff_ra), color='k', fontdict=font2)
ax1.text(xlim0 + 0.05, ylim1 - 1.75, ' $\Delta$ Dec (mas) = %5.3f $\pm$ %5.3f'
         % (med_diff_dec, sig_diff_dec), color='k', fontdict=font2)

plt.tight_layout()

filename = 'deltaRA_deltaDec_LMCcat_vs_{}_{}.png'.format(det, filt2)

#plt.savefig(os.path.join(figures_dir, filename))

8.<font color='white'>-</font>Dither offset check<a class="anchor" id="dither"></a>
------------------

We check using 1 filter (F200W in this particular case) if the dither offsets are correct.

In [ ]:
filt_check_dither = 'F200W'
det_check_dither = 'NRCB1'

image_1 = dict_images[det_check_dither][filt_check_dither]['images'][0]
image_2 = dict_images[det_check_dither][filt_check_dither]['images'][1] 

catalog_1 = results_final_2[0]
catalog_2 = results_final_2[1]

im_1 = fits.open(image_1)
im_2 = fits.open(image_2)

phead_1 = im_1[0].header
fhead_1 = im_1[1].header

phead_2 = im_2[0].header
fhead_2 = im_2[1].header

### 8.1<font color='white'>-</font>Cross-match the catalogs - same filter, different dither position<a class="anchor" id="crossmatch_dither"></a> ###

In [ ]:
max_sep = 0.5
pixelscale = 0.031 # select the right pixelscale for SW or LW detector

match_phot_1 = Table()
match_phot_2 = Table()

idx, d2d, _ = match_coordinates_sky(catalog_1['radec'], catalog_2['radec'])
sep_constraint = (d2d.arcsec / pixelscale) < max_sep
    
match_phot_1 = catalog_1[sep_constraint]
match_phot_2 = catalog_2[idx[sep_constraint]]

print('Number of stars in common between the two filter {} catalogs:'.format(filt_check_dither), 
      len(match_phot_1))

### 8.2<font color='white'>-</font>Convert sky coordinates to Ideal coordinates <a class="anchor" id="convert_dither"></a> ###

In [ ]:
def convert_to_idl(ra, dec, phdr, fhdr):

    instrument = phdr['INSTRUME']
    apername = phdr['APERNAME']
    
    ra_ref_orig = fhdr['RA_REF']
    dec_ref_orig = fhdr['DEC_REF']
    roll_ref_orig = fhdr['ROLL_REF']
    
    siaf = pysiaf.Siaf(instrument)

    aper_orig = siaf[apername]

    v2_ref_orig = aper_orig.V2Ref     
    v3_ref_orig = aper_orig.V3Ref     

    attitude = pysiaf.utils.rotations.attitude(v2_ref_orig, v3_ref_orig, ra_ref_orig, dec_ref_orig, roll_ref_orig)

    aper_orig.set_attitude_matrix(attitude)

    aper_NRCALL_FULL = siaf['NRCALL_FULL']
    aper_NRCALL_FULL.set_attitude_matrix(attitude)

    x_idl, y_idl      = aper_NRCALL_FULL.convert(ra, dec, 'sky', 'idl')
    
    return x_idl, y_idl

In [ ]:
# 1st catalog 

ra_1 = match_phot_1['radec'].ra.degree
dec_1 = match_phot_1['radec'].dec.degree

x_Idl_1 = []
y_Idl_1 = []

for ra, dec in zip(ra_1, dec_1):
    
    x_i, y_i = convert_to_idl(ra, dec, phead_1, fhead_1)
    
    x_Idl_1.append(x_i)
    y_Idl_1.append(y_i)

xy_Idl_1 = Table()

xy_Idl_1['x_Idl'] = x_Idl_1
xy_Idl_1['y_Idl'] = y_Idl_1

In [ ]:
# 2nd catalog

ra_2 = match_phot_2['radec'].ra.degree
dec_2 = match_phot_2['radec'].dec.degree

x_Idl_2 = []
y_Idl_2 = []

for ra, dec in zip(ra_2, dec_2):
    
    x_i, y_i = convert_to_idl(ra, dec, phead_2, fhead_2)
    
    x_Idl_2.append(x_i)
    y_Idl_2.append(y_i)

xy_Idl_2 = Table()

xy_Idl_2['x_Idl'] = x_Idl_2
xy_Idl_2['y_Idl'] = y_Idl_2

### 8.3<font color='white'>-</font>Plot offset and offset corrected (using header information)<a class="anchor" id="offset_dither"></a> ###

In [ ]:
plt.figure(figsize=(14, 8))

ax1 = plt.subplot(1, 2, 1)

ax1.set_xlabel('$\Delta$ X$_{Idl}$ [arcsec]', fontdict=font2)
ax1.set_ylabel('$\Delta$ Y$_{Idl}$ [arcsec]', fontdict=font2)

ax1.set_title('$\Delta$ (XY)$_{Idl}$ difference', fontdict=font2)

plt.suptitle(det_check_dither + ' - '+ filt_check_dither, fontsize=30)

diff_x = xy_Idl_1['x_Idl'] - xy_Idl_2['x_Idl']
diff_y = xy_Idl_1['y_Idl'] - xy_Idl_2['y_Idl'] 

_, med_diff_x, sig_diff_x = sigma_clipped_stats(diff_x)
_, med_diff_y, sig_diff_y = sigma_clipped_stats(diff_y)

xlim0 = med_diff_x - 0.2
xlim1 = med_diff_x + 0.2
ylim0 = med_diff_y - 0.2
ylim1 = med_diff_y + 0.2

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.scatter(diff_x, diff_y, s=5, color='k')
ax1.plot([med_diff_x, med_diff_x], [ylim0, ylim1], color='k', lw=2, ls='--')
ax1.plot([xlim0, xlim1], [med_diff_y, med_diff_y], color='k', lw=2, ls='--')

ax1.text(xlim0 + 0.01, ylim1 - 0.03, ' $\Delta$ X_Idl (arcsec) = %6.4f $\pm$ %5.4f'
         % (med_diff_x, sig_diff_x), color='k', fontdict=font2)
ax1.text(xlim0 + 0.01, ylim1 - 0.06, ' $\Delta$ Y_Idl (arcsec) = %6.4f $\pm$ %5.4f'
         % (med_diff_y, sig_diff_y), color='k', fontdict=font2)

ax2 = plt.subplot(1, 2, 2)

ax2.set_xlabel('$\Delta$ X$_{Idl}$ + offset [arcsec]', fontdict=font2)
ax2.set_ylabel('$\Delta$ Y$_{Idl}$ + offset [arcsec]', fontdict=font2)

ax2.set_title('$\Delta$ (XY)$_{Idl}$ after applying offset', fontdict=font2)

diff_x_corr = xy_Idl_1['x_Idl'] - xy_Idl_2['x_Idl'] + phead_2['XOFFSET']
diff_y_corr = xy_Idl_1['y_Idl'] - xy_Idl_2['y_Idl'] + phead_2['YOFFSET']

_, med_diff_x_corr, sig_diff_x_corr = sigma_clipped_stats(diff_x_corr)
_, med_diff_y_corr, sig_diff_y_corr = sigma_clipped_stats(diff_y_corr)

xlim0 = -0.2
xlim1 = 0.2
ylim0 = -0.2
ylim1 = 0.2

ax2.set_xlim(xlim0, xlim1)
ax2.set_ylim(ylim0, ylim1)

ax2.xaxis.set_major_locator(ticker.AutoLocator())
ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax2.yaxis.set_major_locator(ticker.AutoLocator())
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())


ax2.scatter(diff_x_corr, diff_y_corr, s=5, color='k')
ax2.plot([0, 0], [ylim0, ylim1], color='k', lw=2, ls='--')
ax2.plot([xlim0, xlim1], [0, 0], color='k', lw=2, ls='--')

ax2.text(xlim0 + 0.01, ylim1 - 0.03, ' Residual X = %6.4f $\pm$ %5.4f'
         % (med_diff_x_corr, sig_diff_x_corr), color='k', fontdict=font2)
ax2.text(xlim0 + 0.01, ylim1 - 0.06, ' Residual Y = %6.4f $\pm$ %5.4f'
         % (med_diff_y_corr, sig_diff_y_corr), color='k', fontdict=font2)

plt.tight_layout()

filename = 'dither_offset_{}_{}.png'.format(det_check_dither, filt_check_dither)

#plt.savefig(os.path.join(figures_dir, filename))